In [ ]:
class h5File(h5py.File):
    def __init__(self, *args, **kwargs):
        h5py.File.__init__(self, *args, **kwargs)

    def add(self, key, data):
        data = np.array(data)
        try:
            self.create_dataset(key, shape=data.shape,
                                maxshape=tuple([None] * len(data.shape)),
                                dtype=str(data.dtype))
        except RuntimeError:
            del self[key]
            self.create_dataset(key, shape=data.shape,
                                maxshape=tuple([None] * len(data.shape)),
                                dtype=str(data.dtype))
        self[key][...] = data

    def append(self, key, data, forceInit=False):
        data = np.array(data)
        try:
            self.create_dataset(key, shape=tuple([1] + list(data.shape)),
                                maxshape=tuple([None] * (len(data.shape) + 1)),
                                dtype=str(data.dtype))
        except RuntimeError:
            if forceInit == True:
                del self[key]
                self.create_dataset(key, shape=tuple([1] + list(data.shape)),
                                    maxshape=tuple([None] * (len(data.shape) + 1)),
                                    dtype=str(data.dtype))
            dataset = self[key]
            Shape = list(dataset.shape)
            Shape[0] = Shape[0] + 1
            dataset.resize(Shape)

        dataset = self[key]
        try:
            dataset[-1, :] = data
        except TypeError:
            dataset[-1] = data
            # Usage require strictly same dimensionality for all data appended.
            # currently I don't have it setup to return a good exception, but should